## **Bank Analysis**
---

- Task : Classification
- Objective : Prediksi client bank yang berlangganan term deposit

<br>

### **Data description:**

**Bank Client Data**:

- `age` (numeric)
- `job` : type of job (categorical: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar", "self-employed", "retired", "technician", "services")
- `marital` : marital status (categorical: "married", "divorced", "single"; note: "divorced" means divorced or widowed)
- `education` (categorical: "unknown", "secondary", "primary", "tertiary")
- `default`: has credit in default? (binary: "yes", "no")
- `balance`: average yearly balance, in euros (numeric)
- `housing`: has housing loan? (binary: "yes", "no")
- `loan`: has personal loan? (binary: "yes", "no")

<br>

**Kondisi komunikasi dengan campaign terakhir**
- `contact`: contact communication type (categorical: "unknown", "telephone", "cellular")
- `day`: last contact day of the month (numeric)
- `month`: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
- `duration`: last contact duration, in seconds (numeric)

<br>

**Atribut/Fitur lain**
- `campaign`: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
- `previous`: number of contacts performed before this campaign and for this client (numeric)
- `poutcome`: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

<br>

**Output variable (desired target)**
- `y` - has the client subscribed a term deposit? (binary: "yes","no")

## <b><font color='blue'>1.  Importing Data to Python</font></b>
---

Anda dapat import data dari berbagai format:
- .csv
- .Excel
- .txt
- .SQL
- .dat

**Import library pengolahan data**

Biasanya
- Pandas
- Numpy

In [1]:
import pandas as pd
import numpy as np

**Load data**

- Pakai `pd.read_csv()` apabila file-nya .csv
- Load data `bank-data.csv`

In [2]:
bank_df = pd.read_csv('source/bank-data.csv')

In [3]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,NaN,NaN,261.0,1.0,-1.0,0.0,unknown,no
1,NaN,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,NaN,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no
3,NaN,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown,no
4,33.0,unknown,single,unknown,no,1.0,no,no,NaN,5.0,may,198.0,1.0,-1.0,0.0,unknown,no


In [4]:
bank_df.shape

(45211, 17)

**Cek & Drop data yang duplikat**

- cek-nya pakai `.duplicated()`

In [5]:
duplicate_status = bank_df.duplicated()
duplicate_status

0        False
1        False
2        False
3        False
4        False
         ...  
45206    False
45207    False
45208    False
45209    False
45210    False
Length: 45211, dtype: bool

- cari jumlahnya pakai `.sum()`

In [6]:
duplicate_status.sum()

# FALSE = 0 --> kalo tidak duplikat 
# TRUE = 1 --> kalo duplikat
# Kalau ada yang duplikat, maka jumlahnya > 0

0

- Drop data duplicate dengan `.drop_duplicates()`

In [8]:
bank_df = bank_df.drop_duplicates()

In [9]:
bank_df.shape

(45211, 17)

**Buat semuanya dalam fungsi**

1. Import data
2. Cek **Jumlah observasi** dan **Jumlah kolom**
3. Drop duplicate
4. Cek **Jumlah observasi** dan **Jumlah kolom** setelah di-drop
5. Return data setelah di-drop

In [11]:
bank_df = pd.read_csv("source/bank-data.csv")
print("Data asli            : ", bank_df.shape, "- (#observasi, #kolom)")

bank_df = bank_df.drop_duplicates()
print("Data setelah di-drop : ", bank_df.shape, "- (#observasi, #kolom)")

Data asli            :  (45211, 17) - (#observasi, #kolom)
Data setelah di-drop :  (45211, 17) - (#observasi, #kolom)


In [12]:
def importData(filename):
    """
    Fungsi untuk import data & hapus duplikat
    :param filename: <string> nama file input (format .csv)
    :return df: <pandas dataframe> sampel data
    """

    # read data
    df = pd.read_csv(filename)
    print("Data asli            : ", df.shape, "- (#observasi, #kolom)")

    # drop duplicates
    df = df.drop_duplicates()
    print("Data setelah di-drop : ", df.shape, "- (#observasi, #kolom)")

    return df

In [13]:
# input
file_bank = "source/bank-data.csv"

# panggil fungsi
bank_df = importData(filename = file_bank)

Data asli            :  (45211, 17) - (#observasi, #kolom)
Data setelah di-drop :  (45211, 17) - (#observasi, #kolom)


In [14]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,NaN,NaN,261.0,1.0,-1.0,0.0,unknown,no
1,NaN,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,NaN,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no
3,NaN,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown,no
4,33.0,unknown,single,unknown,no,1.0,no,no,NaN,5.0,may,198.0,1.0,-1.0,0.0,unknown,no


## <b><font color='blue'> 2. Data Preprocessing:</font></b>
---
    * Input-Output Split, Train-Test Split
    * Processing Categorical
    * Imputation, Normalization, Drop Duplicates

### **Input-Output Split**

- Fitur `y` adalah output variabel dari data marketing
- yang lainnya menjadi input

In [15]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,NaN,NaN,261.0,1.0,-1.0,0.0,unknown,no
1,NaN,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,NaN,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no
3,NaN,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown,no
4,33.0,unknown,single,unknown,no,1.0,no,no,NaN,5.0,may,198.0,1.0,-1.0,0.0,unknown,no


In [16]:
output_data = bank_df["y"]

In [17]:
output_data.head()

0    no
1    no
2    no
3    no
4    no
Name: y, dtype: object

**Buat data input**

- DATA = INPUT + OUTPUT
- DATA - OUTPUT = INPUT
- Jadi kalau dari data, kita drop VARIABLE OUTPUT, maka tersisa hanya variabel INPUT.

In [18]:
input_data = bank_df.drop(["y"], 
                          axis = 1)

In [19]:
input_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,NaN,NaN,261.0,1.0,-1.0,0.0,unknown
1,NaN,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown
2,NaN,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown
3,NaN,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown
4,33.0,unknown,single,unknown,no,1.0,no,no,NaN,5.0,may,198.0,1.0,-1.0,0.0,unknown


**Buat semuanya jadi fungsi**
1. buat output_data
2. buat input_data
3. return input_data dan output_data

In [21]:
output_data = bank_df["y"]
input_data = bank_df.drop("y", axis = 1)

In [22]:
def extractInputOutput(data, output_column_name):
    """
    Fungsi untuk memisahkan data input dan output
    :param data: <pandas dataframe> data seluruh sample
    :param output_column_name: <string> nama kolom output
    :return input_data: <pandas dataframe> data input
    :return output_data: <pandas series> data output
    """
    output_data = data[output_column_name]
    input_data = data.drop(output_column_name,
                           axis = 1)
    
    return input_data, output_data

# (data, output_column_name) adalah argumen
# Argumen adalah sebuah variable. 
# Jika fungsi tsb. diberi argumen data = bank_df, 
# maka semua variabel 'data' di dalam fungsi akan berubah menjadi bank_df

In [23]:
X, y = extractInputOutput(data = bank_df, output_column_name = "y")

In [24]:
X.head(2)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,NaN,NaN,261.0,1.0,-1.0,0.0,unknown
1,NaN,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown


In [25]:
y.head(2)

0    no
1    no
Name: y, dtype: object

### **Train-Test Split**

- **Kenapa?**
  - Karena tidak mau overfit data training
  - Test data akan menjadi future data
  - Kita akan latih model ML di data training, dengan CV (Cross-validation)
  - Selanjutnya melakukan evaluasi di data testing

In [26]:
from sklearn.model_selection import train_test_split

**Train Test Split Function**
1. `X` adalah input
2. `y` adalah output (target)
3. `test_size` adalah seberapa besar proporsi data test dari keseluruhan data. Contoh `test_size = 0.2` artinya data test akan berisi 20% data.
4. `random_state` adalah kunci untuk random. Harus di-setting sama. Misal `random_state = 123`.
5. Output:
   - `X_train` = input dari data training
   - `X_test` = input dari data testing
   - `y_train` = output dari data training
   - `y_test` = output dari data testing
6. Urutan outputnya: `X_train, X_test, y_train, y_test`. Tidak boleh terbalik

> Readmore: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [27]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.25,
                                                    random_state = 12)

In [28]:
# Sanity check hasil splitting
print(X_train.shape)
print(X_test.shape)

(33908, 16)
(11303, 16)


In [29]:
# Ratio
X_test.shape[0] / X.shape[0]

# Hasil 0.25 - sesuai dengan test_size kita

0.25000552962774547

### **Data Imputation**

- Proses pengisian data yang kosong (NaN)
- Ada 2 hal yang diperhatikan:
  - Numerical Imputation
  - Categorical Imputation

In [30]:
X_train.isnull().sum()

# Output: nama variabel, True/False.
# Jika True, maka ada data yang kosong

age          2626
job          2650
marital      2650
education    2542
default      2689
balance      2574
housing      2660
loan         2668
contact      2695
day          2617
month        2602
duration     2701
campaign     2614
pdays        2634
previous     2638
poutcome     2629
dtype: int64

**Bedakan antara data categorical & numerical**

In [31]:
X_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
37156,35.0,management,single,tertiary,no,2749.0,no,no,cellular,13.0,may,127.0,1.0,-1.0,0.0,unknown
20494,30.0,management,NaN,NaN,no,443.0,yes,NaN,cellular,12.0,NaN,80.0,2.0,-1.0,0.0,unknown
35272,39.0,management,NaN,tertiary,no,4239.0,yes,no,cellular,7.0,may,40.0,1.0,-1.0,0.0,unknown
22260,49.0,services,NaN,NaN,no,400.0,no,no,cellular,21.0,aug,151.0,3.0,-1.0,0.0,unknown
2728,28.0,technician,single,secondary,no,468.0,yes,no,unknown,13.0,may,152.0,3.0,-1.0,0.0,unknown


Data kategorikal:
- job
- marital
- education
- default
- housing
- loan
- contact
- month
- poutcome

Sisanya adalah numerical

In [32]:
X_train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome'],
      dtype='object')

In [33]:
numerical_column = ["age", "balance", "day", "duration", 
                    "campaign", "pdays", "previous"]

In [34]:
# Seleksi dataframe numerik
X_train_numerical = X_train[numerical_column]

In [35]:
X_train_numerical.head()

,age,balance,day,duration,campaign,pdays,previous
37156,35.0,2749.0,13.0,127.0,1.0,-1.0,0.0
20494,30.0,443.0,12.0,80.0,2.0,-1.0,0.0
35272,39.0,4239.0,7.0,40.0,1.0,-1.0,0.0
22260,49.0,400.0,21.0,151.0,3.0,-1.0,0.0
2728,28.0,468.0,13.0,152.0,3.0,-1.0,0.0


In [36]:
X_train_numerical.isnull().any()

# Semua variabel numerical memiliki missing values

age         True
balance     True
day         True
duration    True
campaign    True
pdays       True
previous    True
dtype: bool

In [37]:
from sklearn.impute import SimpleImputer

In [38]:
imputer = SimpleImputer(missing_values = np.nan,
                        strategy = "median")

# namakan function SimpleImputer menjadi imputer, jangan lupa tanda kurung ()
# missing_values adalah tanda missing values dalam data.
#   - bisa NaN, bisa 999, bisa "KOSONG"
# Strategy median adalah strategy imputasi, 
# jika data kosong, diganti dengan median target
# Strategi lainnya adalah: mean

- `fit` : imputer agar mengetahui mean atau median dari tiap kolom
- `transform` : isi data dengan median atau mean
- output dari transform adalah pandas dataframe
- namakan kolom `X_train_numerical_imputed` sesuai dengan `X_train_numerical`.
   - MENGAPA? karena kita kehilangan nama kolom setelah data imputation
- beri index dari `X_train_numerical_imputed` sesuai dengan `X_train_numerical`.
   - MENGAPA? karena kita kehilangan index setelah data imputation

In [39]:
# Fit imputer
imputer.fit(X_train_numerical)

# Transform
imputed_data = imputer.transform(X_train_numerical)
X_train_numerical_imputed = pd.DataFrame(imputed_data)

X_train_numerical_imputed.columns = X_train_numerical.columns
X_train_numerical_imputed.index = X_train_numerical.index

In [40]:
X_train_numerical_imputed.isnull().any()

age         False
balance     False
day         False
duration    False
campaign    False
pdays       False
previous    False
dtype: bool

In [41]:
from sklearn.impute import SimpleImputer

def numericalImputation(data, numerical_column):
    """
    Fungsi untuk melakukan imputasi data numerik
    :param data: <pandas dataframe> sample data input
    :param numerical_column: <list> list kolom numerik data
    :return X_train_numerical: <pandas dataframe> data numerik
    :return imputer_numerical: numerical imputer method
    """
    # Filter data numerik
    numerical_data = data[numerical_column]

    # Buat imputer
    imputer_numerical = SimpleImputer(missing_values = np.nan,
                                      strategy = "median")
    imputer_numerical.fit(numerical_data)

    # Transform
    imputed_data = imputer_numerical.transform(numerical_data)
    numerical_data_imputed = pd.DataFrame(imputed_data)

    numerical_data_imputed.columns = numerical_column
    numerical_data_imputed.index = numerical_data.index

    return numerical_data_imputed, imputer_numerical

In [42]:
# Input
numerical_column = ["age", "balance", "day", "duration", 
                    "campaign", "pdays", "previous"]

# Imputation Numeric
X_train_numerical, imputer_numerical = numericalImputation(data = X_train,
                                                           numerical_column = numerical_column)

In [43]:
X_train_numerical.isnull().any()

age         False
balance     False
day         False
duration    False
campaign    False
pdays       False
previous    False
dtype: bool

In [44]:
# Ambil daftar nama kolom kategorikal
# Anda bisa langsung menuliskannya atau mengambil list jika jumlahnya banyak

X_train_column = list(X_train.columns)
categorical_column = list(set(X_train_column).difference(set(numerical_column)))

In [45]:
categorical_column

['loan',
 'default',
 'poutcome',
 'contact',
 'month',
 'marital',
 'job',
 'education',
 'housing']

In [46]:
categorical_data = X_train[categorical_column]
categorical_data.isnull().sum()

loan         2668
default      2689
poutcome     2629
contact      2695
month        2602
marital      2650
job          2650
education    2542
housing      2660
dtype: int64

In [47]:
# Kita isi kolom kategorik dengan "KOSONG"
categorical_data = X_train[categorical_column]
categorical_data = categorical_data.fillna(value="KOSONG")

In [48]:
categorical_data.isnull().sum()

loan         0
default      0
poutcome     0
contact      0
month        0
marital      0
job          0
education    0
housing      0
dtype: int64

In [49]:
def categoricalImputation(data, categorical_column):
    """
    Fungsi untuk melakukan imputasi data kategorik
    :param data: <pandas dataframe> sample data input
    :param categorical_column: <list> list kolom kategorikal data
    :return categorical_data: <pandas dataframe> data kategorikal
    """
    # seleksi data
    categorical_data = data[categorical_column]

    # lakukan imputasi
    categorical_data = categorical_data.fillna(value="KOSONG")

    return categorical_data


In [50]:
X_train_categorical = categoricalImputation(data = X_train,
                                            categorical_column = categorical_column)

In [51]:
X_train_categorical.isnull().sum()

loan         0
default      0
poutcome     0
contact      0
month        0
marital      0
job          0
education    0
housing      0
dtype: int64

### **Preprocessing Categorical Variables**

- Kita tidak bisa memasukkan data categorical, jika tidak diubah menjadi numerical
- Solusi: One Hot Encoding (OHE)

In [52]:
categorical_ohe = pd.get_dummies(X_train_categorical)

In [53]:
categorical_ohe.head(2)

,loan_KOSONG,loan_no,loan_yes,default_KOSONG,default_no,default_yes,poutcome_KOSONG,poutcome_failure,poutcome_other,poutcome_success,...,job_unemployed,job_unknown,education_KOSONG,education_primary,education_secondary,education_tertiary,education_unknown,housing_KOSONG,housing_no,housing_yes
37156,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
20494,True,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [54]:
def extractCategorical(data, categorical_column):
    """
    Fungsi untuk ekstrak data kategorikal dengan One Hot Encoding
    :param data: <pandas dataframe> data sample
    :param categorical_column: <list> list kolom kategorik
    :return categorical_ohe: <pandas dataframe> data sample dengan ohe
    """
    data_categorical = categoricalImputation(data = data,
                                             categorical_column = categorical_column)
    categorical_ohe = pd.get_dummies(data_categorical)

    return categorical_ohe

In [55]:
X_train_categorical_ohe = extractCategorical(data = X_train,
                                             categorical_column = categorical_column)

In [56]:
X_train_categorical_ohe.head()

,loan_KOSONG,loan_no,loan_yes,default_KOSONG,default_no,default_yes,poutcome_KOSONG,poutcome_failure,poutcome_other,poutcome_success,...,job_unemployed,job_unknown,education_KOSONG,education_primary,education_secondary,education_tertiary,education_unknown,housing_KOSONG,housing_no,housing_yes
37156,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
20494,True,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
35272,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
22260,False,True,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
2728,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [57]:
# Simpan kolom OHE untuk diimplementasikan dalam testing data
# Agar shape-nya konsisten
ohe_columns = X_train_categorical_ohe.columns

In [58]:
ohe_columns

Index(['loan_KOSONG', 'loan_no', 'loan_yes', 'default_KOSONG', 'default_no',
       'default_yes', 'poutcome_KOSONG', 'poutcome_failure', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown', 'contact_KOSONG',
       'contact_cellular', 'contact_telephone', 'contact_unknown',
       'month_KOSONG', 'month_apr', 'month_aug', 'month_dec', 'month_feb',
       'month_jan', 'month_jul', 'month_jun', 'month_mar', 'month_may',
       'month_nov', 'month_oct', 'month_sep', 'marital_KOSONG',
       'marital_divorced', 'marital_married', 'marital_single', 'job_KOSONG',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'education_KOSONG', 'education_primary', 'education_secondary',
       'education_tertiary', 'education_unknown', 'housing_KOSONG',
       'housing_no', 'housing_yes'],
      dtype='object'

### **Join data Numerical dan Categorical**

- Data numerik & kategorik harus disatukan kembali
- Penyatuan dengan `pd.concat`

In [59]:
X_train_concat = pd.concat([X_train_numerical,
                            X_train_categorical_ohe],
                           axis = 1)

In [60]:
X_train_concat.head()

,age,balance,day,duration,campaign,pdays,previous,loan_KOSONG,loan_no,loan_yes,...,job_unemployed,job_unknown,education_KOSONG,education_primary,education_secondary,education_tertiary,education_unknown,housing_KOSONG,housing_no,housing_yes
37156,35.0,2749.0,13.0,127.0,1.0,-1.0,0.0,False,True,False,...,False,False,False,False,False,True,False,False,True,False
20494,30.0,443.0,12.0,80.0,2.0,-1.0,0.0,True,False,False,...,False,False,True,False,False,False,False,False,False,True
35272,39.0,4239.0,7.0,40.0,1.0,-1.0,0.0,False,True,False,...,False,False,False,False,False,True,False,False,False,True
22260,49.0,400.0,21.0,151.0,3.0,-1.0,0.0,False,True,False,...,False,False,True,False,False,False,False,False,True,False
2728,28.0,468.0,13.0,152.0,3.0,-1.0,0.0,False,True,False,...,False,False,False,False,True,False,False,False,False,True


In [61]:
X_train_concat.isnull().any()

age                    False
balance                False
day                    False
duration               False
campaign               False
pdays                  False
previous               False
loan_KOSONG            False
loan_no                False
loan_yes               False
default_KOSONG         False
default_no             False
default_yes            False
poutcome_KOSONG        False
poutcome_failure       False
poutcome_other         False
poutcome_success       False
poutcome_unknown       False
contact_KOSONG         False
contact_cellular       False
contact_telephone      False
contact_unknown        False
month_KOSONG           False
month_apr              False
month_aug              False
month_dec              False
month_feb              False
month_jan              False
month_jul              False
month_jun              False
month_mar              False
month_may              False
month_nov              False
month_oct              False
month_sep     

### **Standardizing Variables**

- Menyamakan skala dari variabel input
- `fit`: imputer agar mengetahui mean dan standar deviasi dari setiap kolom
- `transform`: isi data dengan value yang sudah dinormalisasi
- output dari transform berupa pandas dataframe
- normalize dikeluarkan karena akan digunakan pada data test

In [62]:
from sklearn.preprocessing import StandardScaler

# Buat fungsi
def standardizerData(data):
    """
    Fungsi untuk melakukan standarisasi data
    :param data: <pandas dataframe> sampel data
    :return standardized_data: <pandas dataframe> sampel data standard
    :return standardizer: method untuk standardisasi data
    """
    data_columns = data.columns  # agar nama kolom tidak hilang
    data_index = data.index  # agar index tidak hilang

    # buat (fit) standardizer
    standardizer = StandardScaler()
    standardizer.fit(data)

    # transform data
    standardized_data_raw = standardizer.transform(data)
    standardized_data = pd.DataFrame(standardized_data_raw)
    standardized_data.columns = data_columns
    standardized_data.index = data_index

    return standardized_data, standardizer

In [63]:
X_train_clean, standardizer = standardizerData(data = X_train_concat)

In [64]:
X_train_clean.head()

,age,balance,day,duration,campaign,pdays,previous,loan_KOSONG,loan_no,loan_yes,...,job_unemployed,job_unknown,education_KOSONG,education_primary,education_secondary,education_tertiary,education_unknown,housing_KOSONG,housing_no,housing_yes
37156,-0.568886,0.502047,-0.354434,-0.504073,-0.569761,-0.390255,-0.292138,-0.292238,0.543266,-0.418762,...,-0.165313,-0.07566,-0.284681,-0.402775,-0.949599,1.632949,-0.200227,-0.291763,1.201004,-1.024543
20494,-1.058043,-0.288093,-0.479210,-0.693995,-0.236736,-0.390255,-0.292138,3.421863,-1.840720,-0.418762,...,-0.165313,-0.07566,3.512706,-0.402775,-0.949599,-0.612389,-0.200227,-0.291763,-0.832636,0.976045
35272,-0.177561,1.012588,-1.103089,-0.855631,-0.569761,-0.390255,-0.292138,-0.292238,0.543266,-0.418762,...,-0.165313,-0.07566,-0.284681,-0.402775,-0.949599,1.632949,-0.200227,-0.291763,-0.832636,0.976045
22260,0.800752,-0.302827,0.643772,-0.407091,0.096289,-0.390255,-0.292138,-0.292238,0.543266,-0.418762,...,-0.165313,-0.07566,3.512706,-0.402775,-0.949599,-0.612389,-0.200227,-0.291763,1.201004,-1.024543
2728,-1.253705,-0.279527,-0.354434,-0.403050,0.096289,-0.390255,-0.292138,-0.292238,0.543266,-0.418762,...,-0.165313,-0.07566,-0.284681,-0.402775,1.053076,-0.612389,-0.200227,-0.291763,-0.832636,0.976045


## <b><font color='blue'> 3. Training Machine Learning:</font></b>
---
    * Choose Score to optimize and Hyperparameter Space
    * Cross-Validation: Random vs Grid Search CV
    * Kita harus mengalahkan benchmark

### **Benchmark / Baseline**

- Baseline untuk evaluasi nanti
- Karena ini klasifikasi, bisa kita ambil dari proporsi kelas target yang terbesar
- Dengan kata lain, menebak hasil output marketing response dengan nilai "no" semua tanpa modeling

In [65]:
y_train.value_counts(normalize = True)

# baseline akurasi = 88%

y
no     0.882624
yes    0.117376
Name: proportion, dtype: float64

### **1. Import Model**

- Misal kita gunakan 3 model ML untuk klasifikasi:
    - K-nearest neighbor (K-NN)
    - Logistic Regression
    - Random Forest

In [66]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

### **2. Fitting Model**

- Cara fitting/training model mengikuti yang dokumentasi model

In [67]:
# Model K nearest neighbor
knn = KNeighborsClassifier()
knn.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [68]:
# Model Logistic Regression
logreg = LogisticRegression(random_state = 123)
logreg.fit(X_train_clean, y_train)

LogisticRegression(random_state=123)

In [69]:
# Model Random Forest Classifier
random_forest = RandomForestClassifier(random_state = 123)
random_forest.fit(X_train_clean, y_train)

RandomForestClassifier(random_state=123)

In [70]:
# Model Random Forest Classifier 1
# Mari kita ubah hyperparameter dari random forest --> n_estimator
# Maksud & tujuan akan dijelaskan pada kelas Random Forest
# Tambahkan n_estimator = 500

random_forest_1 = RandomForestClassifier(random_state = 123,
                                         n_estimators = 500)
random_forest_1.fit(X_train_clean, y_train)

RandomForestClassifier(n_estimators=500, random_state=123)

### **3. Prediction**

- Saatnya melakukan prediksi

In [71]:
# Prediksi Logistic Regression
logreg.predict(X_train_clean)

array(['no', 'no', 'no', ..., 'no', 'no', 'yes'], dtype=object)

In [72]:
predicted_logreg = pd.DataFrame(logreg.predict(X_train_clean))
predicted_logreg

,0
0,no
1,no
2,no
3,no
4,no
...,...
33903,no
33904,no
33905,no
33906,no


In [73]:
predicted_knn = pd.DataFrame(knn.predict(X_train_clean))
predicted_knn.head()

,0
0,no
1,no
2,no
3,no
4,no


In [74]:
predicted_rf = pd.DataFrame(random_forest.predict(X_train_clean))
predicted_rf.head()

,0
0,no
1,no
2,no
3,no
4,no


In [75]:
predicted_rf_1 = pd.DataFrame(random_forest_1.predict(X_train_clean))
predicted_rf_1.head()

,0
0,no
1,no
2,no
3,no
4,no


### **4. Cek performa model di data training**

In [77]:
benchmark = y_train.value_counts(normalize=True)[0]
benchmark

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16884\4089324966.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  benchmark = y_train.value_counts(normalize=True)[0]


0.8826235696590775

In [78]:
# akurasi knn
knn.score(X_train_clean, y_train)

0.9098737761000354

In [79]:
# akurasi logistic regression
logreg.score(X_train_clean, y_train)

0.900554441429751

In [80]:
# akurasi random forest
random_forest.score(X_train_clean, y_train)

1.0

In [81]:
# akurasi random forest 1
random_forest_1.score(X_train_clean, y_train)

1.0

### **5. Simpan model ke file pickle**

In [82]:
import joblib

# Simpan model logreg ke dalam folder yang sama dengan notebook
# dengan nama logreg.pkl
joblib.dump(logreg, "logreg.pkl")

joblib.dump(knn, "knn.pkl")
joblib.dump(random_forest, "random_forest.pkl")
joblib.dump(random_forest_1, "random_forest_1.pkl")

['random_forest_1.pkl']

### **6. Test Prediction**

1. Siapkan file test dataset
2. Lakukan preprocessing yang sama dengan yang dilakukan di train dataset
3. gunakan `imputer_numerical` dan `standardizer` yang telah di-fit di train dataset

In [83]:
def extractTest(data,
                numerical_column, categorical_column, ohe_column,
                imputer_numerical, standardizer):
    """
    Fungsi untuk mengekstrak & membersihkan test data 
    :param data: <pandas dataframe> sampel data test
    :param numerical_column: <list> kolom numerik
    :param categorical_column: <list> kolom kategorik
    :param ohe_column: <list> kolom one-hot-encoding dari data kategorik
    :param imputer_numerical: <sklearn method> imputer data numerik
    :param standardizer: <sklearn method> standardizer data
    :return cleaned_data: <pandas dataframe> data final
    """
    # Filter data
    numerical_data = data[numerical_column]
    categorical_data = data[categorical_column]

    # Proses data numerik
    numerical_data = pd.DataFrame(imputer_numerical.transform(numerical_data))
    numerical_data.columns = numerical_column
    numerical_data.index = data.index

    # Proses data kategorik
    categorical_data = categorical_data.fillna(value="KOSONG")
    categorical_data.index = data.index
    categorical_data = pd.get_dummies(categorical_data)
    categorical_data.reindex(index = categorical_data.index, 
                             columns = ohe_column)

    # Gabungkan data
    concat_data = pd.concat([numerical_data, categorical_data],
                             axis = 1)
    cleaned_data = pd.DataFrame(standardizer.transform(concat_data))
    cleaned_data.columns = concat_data.columns

    return cleaned_data


In [84]:
def testPrediction(X_test, y_test, classifier, compute_score):
    """
    Fungsi untuk mendapatkan prediksi dari model
    :param X_test: <pandas dataframe> input
    :param y_test: <pandas series> output/target
    :param classifier: <sklearn method> model klasifikasi
    :param compute_score: <bool> True: menampilkan score, False: tidak
    :return test_predict: <list> hasil prediksi data input
    :return score: <float> akurasi model
    """
    if compute_score:
        score = classifier.score(X_test, y_test)
        print(f"Accuracy : {score:.4f}")

    test_predict = classifier.predict(X_test)

    return test_predict, score

In [85]:
X_test_clean = extractTest(data = X_test,
                           numerical_column = numerical_column,
                           categorical_column = categorical_column,
                           ohe_column = ohe_columns,
                           imputer_numerical = imputer_numerical,
                           standardizer = standardizer)

In [86]:
X_test_clean.shape

(11303, 60)

In [87]:
# Logistic Regression Performance
logreg_test_predict, score = testPrediction(X_test = X_test_clean,
                                            y_test = y_test,
                                            classifier = logreg,
                                            compute_score = True)

Accuracy : 0.9010


In [88]:
# K nearest neighbor Performance
knn_test_predict, score = testPrediction(X_test = X_test_clean,
                                         y_test = y_test,
                                         classifier = knn,
                                         compute_score = True)

Accuracy : 0.8898


In [89]:
# Random Forest Performance
rf_test_predict, score = testPrediction(X_test = X_test_clean,
                                        y_test = y_test,
                                        classifier = random_forest,
                                        compute_score = True)

Accuracy : 0.9037


In [90]:
# Random Forest 1 Performance
rf_1_test_predict, score = testPrediction(X_test = X_test_clean,
                                          y_test = y_test,
                                          classifier = random_forest_1,
                                          compute_score = True)  

Accuracy : 0.9021
